# Prepare data with SageMaker Processing

## Setup environment

In [1]:
import os
import boto3
import sagemaker
from sagemaker.processing import Processor, ProcessingInput, ProcessingOutput

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role() # we are using the notebook instance role for training in this example
bucket = sagemaker_session.default_bucket() # you can specify a bucket name here

## Get data

In [2]:
!wget https://aws-mlops-workshop.s3-eu-west-1.amazonaws.com/reviews/workshop_data/reviews.csv

--2020-06-26 21:41:27--  https://aws-mlops-workshop.s3-eu-west-1.amazonaws.com/reviews/workshop_data/reviews.csv
Resolving aws-mlops-workshop.s3-eu-west-1.amazonaws.com (aws-mlops-workshop.s3-eu-west-1.amazonaws.com)... 52.218.105.187
Connecting to aws-mlops-workshop.s3-eu-west-1.amazonaws.com (aws-mlops-workshop.s3-eu-west-1.amazonaws.com)|52.218.105.187|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4445279 (4.2M) [text/csv]
Saving to: ‘reviews.csv’

reviews.csv         100%[===================>]   4.24M  7.93MB/s    in 0.5s    

2020-06-26 21:41:28 (7.93 MB/s) - ‘reviews.csv’ saved [4445279/4445279]



In [3]:
prefix = 'data/input'
s3_input = sagemaker_session.upload_data('reviews.csv', bucket, prefix)
print(s3_input)

s3://sagemaker-us-east-1-175748383800/data/input/reviews.csv


## Build and push container

In [4]:
image_name = 'data-processing-containers'

In [5]:
!sh ./docker/build_and_push.sh $image_name

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Building image with name data-processing-containers
Sending build context to Docker daemon   7.68kB
Step 1/6 : FROM ubuntu:18.04
18.04: Pulling from library/ubuntu

167c320d: Pulling fs layer 
805ec7fd: Pulling fs layer 
d380e680: Pulling fs layer 
Digest: sha256:86510528ab9cd7b64209cbbe6946e094a6d10c6db21def64a93ebdd20011de1d
Status: Downloaded newer image for ubuntu:18.04
 ---> 8e4ce0a6ce69
Step 2/6 : RUN apt-get update && apt-get install -y --no-install-recommends         wget         zip         unzip         git         ca-certificates         curl         python3.6         python3-pip         && rm -rf /var/lib/apt-get/lists/*
 ---> Running in 41bdcdd0905c
Get:1 http://s

Get:38 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libsasl2-modules-db amd64 2.1.27~101-g0780600+dfsg-3ubuntu2.1 [14.8 kB]
Get:39 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libsasl2-2 amd64 2.1.27~101-g0780600+dfsg-3ubuntu2.1 [49.2 kB]
Get:40 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libldap-common all 2.4.45+dfsg-1ubuntu1.5 [16.9 kB]
Get:41 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libldap-2.4-2 amd64 2.4.45+dfsg-1ubuntu1.5 [155 kB]
Get:42 http://archive.ubuntu.com/ubuntu bionic/main amd64 libnghttp2-14 amd64 1.30.0-1ubuntu1 [77.8 kB]
Get:43 http://archive.ubuntu.com/ubuntu bionic/main amd64 librtmp1 amd64 2.4+20151223.gitfa8646d.1-1 [54.2 kB]
Get:44 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libcurl4 amd64 7.58.0-2ubuntu3.9 [214 kB]
Get:45 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 curl amd64 7.58.0-2ubuntu3.9 [159 kB]
Get:46 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libc

Selecting previously unselected package libroken18-heimdal:amd64.
Preparing to unpack .../15-libroken18-heimdal_7.5.0+dfsg-1_amd64.deb ...
Unpacking libroken18-heimdal:amd64 (7.5.0+dfsg-1) ...
Selecting previously unselected package libasn1-8-heimdal:amd64.
Preparing to unpack .../16-libasn1-8-heimdal_7.5.0+dfsg-1_amd64.deb ...
Unpacking libasn1-8-heimdal:amd64 (7.5.0+dfsg-1) ...
Selecting previously unselected package libheimbase1-heimdal:amd64.
Preparing to unpack .../17-libheimbase1-heimdal_7.5.0+dfsg-1_amd64.deb ...
Unpacking libheimbase1-heimdal:amd64 (7.5.0+dfsg-1) ...
Selecting previously unselected package libhcrypto4-heimdal:amd64.
Preparing to unpack .../18-libhcrypto4-heimdal_7.5.0+dfsg-1_amd64.deb ...
Unpacking libhcrypto4-heimdal:amd64 (7.5.0+dfsg-1) ...
Selecting previously unselected package libwind0-heimdal:amd64.
Preparing to unpack .../19-libwind0-heimdal_7.5.0+dfsg-1_amd64.deb ...
Unpacking libwind0-heimdal:amd64 (7.5.0+dfsg-1) ...
Selecting previously unselected pac

 ---> Running in 57fe0b5b40ae
Removing intermediate container 57fe0b5b40ae
 ---> 7f0d6480a889
Step 5/6 : COPY code /opt/program
 ---> 817ce3e6a740
Step 6/6 : ENTRYPOINT ["python3", "/opt/program/prepare_data.py"]
 ---> Running in 984f903a7396
Removing intermediate container 984f903a7396
 ---> bf54221a02a8
Successfully built bf54221a02a8
Successfully tagged data-processing-containers:latest
Pushing image to ECR 175748383800.dkr.ecr.us-east-1.amazonaws.com/data-processing-containers:latest
The push refers to repository [175748383800.dkr.ecr.us-east-1.amazonaws.com/data-processing-containers]

78b93c42: Preparing 
346d4c4c: Preparing 
22dfe983: Preparing 
0d40c713: Preparing 
00d84994: Preparing 
52ea2c16: Preparing 
c9e5703f: Preparing 
346d4c4c: Pushed   223.2MB/217.8MB[6Alatest: digest: sha256:ac210c3a5e1d973cf71546b0857ff150f9edbdafae1ee3264a11ef95ce83d9a3 size: 1990


In [6]:
# Replace by your ECR image ID
container = '175748383800.dkr.ecr.us-east-1.amazonaws.com/data-processing-containers:latest' 

## Launch data processing job

In [7]:
data_processor = Processor(role=role, 
                           image_uri=container, 
                           instance_count=1, 
                           instance_type='ml.m5.xlarge',
                           volume_size_in_gb=30, 
                           max_runtime_in_seconds=1200,
                           base_job_name='data-processing')

In [8]:
input_folder = '/opt/ml/processing/input'
output_folder = '/opt/ml/processing/output'

data_processor.run(
    arguments= [
        f'--input={input_folder}',
        f'--output={output_folder}'
    ],
    inputs = [
        ProcessingInput(
            input_name='input',
            source=s3_input,
            destination=input_folder
        )
    ],
    outputs= [
        ProcessingOutput(
            output_name='preprocessed',
            source=output_folder,
            destination=bucket
        )
    ]
)


Job Name:  data-processing-2020-06-26-21-44-08-917
Inputs:  [{'InputName': 'input', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-175748383800/data/input/reviews.csv', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'preprocessed', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-175748383800/data-processing-2020-06-26-21-44-08-917/output/preprocessed', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
.....................
..